In [ ]:
#| default_exp core

# core

> Fill in a module description here

In [ ]:
#| export
from fastcore.utils import *
from fastcore.xml import *
from fasthtml.common import *
from fasthtml.svg import *
from json import loads
from copy import deepcopy
import pathlib

In [ ]:
__file__='../fastlucide/_icons.py'

In [ ]:
#| export
def read_icons():
    path = pathlib.Path(__file__).parent
    fpath = path/'_icons.py'
    return loads(fpath.read_text())

In [ ]:
icons = read_icons()

In [ ]:
nm = 'a-arrow-down'
ico = icons[nm]
ico

[['path', {'d': 'M3.5 13h6'}],
 ['path', {'d': 'm2 16 4.5-9 4.5 9'}],
 ['path', {'d': 'M18 7v9'}],
 ['path', {'d': 'm14 12 4 4 4-4'}]]

In [ ]:
svg_attrs = dict(fill='none', stroke='currentColor', stroke_width='2', stroke_linecap='round', stroke_linejoin='round')

In [ ]:
#| export
def sz_attrs(sz, vbox=24):
    if isinstance(vbox, int): vbox = (vbox, vbox)
    if isinstance(sz, int): sz = (sz, sz)
    return dict(viewBox=f'0 0 {vbox[0]} {vbox[1]}', width=f'{sz[0]}px', height=f'{sz[1]}px')

In [ ]:
sym = [ft(t, **attrs) for t,attrs in ico]
show(Svg(*sym, **sz_attrs(24), **svg_attrs))
show(Svg(*sym, **sz_attrs(16), **svg_attrs))

In [ ]:
#| export
def symbol(
    icons, # icon dict
    nm, # Name of icon in lucide
    pre='' # Prefix to add to element id
):
    "Create a `symbol` element for an icon"
    ico = icons[nm]
    sym = [ft(t, **attrs) for t,attrs in ico]
    return Symbol(*sym, id=pre+nm)

In [ ]:
symbol(icons, nm, 'l-')

```html
<symbol id="l-a-arrow-down"><path d="M3.5 13h6"></path><path d="m2 16 4.5-9 4.5 9"></path><path d="M18 7v9"></path><path d="m14 12 4 4 4-4"></path></symbol>
```

In [ ]:
#| export
def sprites(
    icons, # icon dict
    nms, # List of lucide icon names
    pre='' # Prefix to add to all element ids
):
    "SVG element containing all symbols in `nms`"
    syms = [symbol(icons, nm, pre) for nm in nms]
    return Svg(Defs(*syms), style="display: none")

In [ ]:
sps = sprites(icons, [nm, 'accessibility'], 'l-')
show(sps)

In [ ]:
#| export
def _style_str(stroke=None, fill=None, stroke_width=None):
    "Build CSS style string from stroke/fill/stroke-width"
    styles = []
    if stroke: styles.append(f'stroke: {stroke}')
    if fill: styles.append(f'fill: {fill}')
    if stroke_width: styles.append(f'stroke-width: {stroke_width}')
    return '; '.join(styles) if styles else None

In [ ]:
#| export
class Icon:
    def __init__(
        self,
        nm, # Name of icon in lucide
        pre='', # Prefix to add to element id
        cls="lucide-icon", # class to use for svg
        vbox=24, # viewBox size (int for square, or (w,h) tuple)
        sz=24, # size of svg
        stroke=None, # Stroke CSS
        stroke_width=None, # Stroke width CSS
        fill=None, # Fill CSS
        trans=None, # SVG transform attribute
        **attrs # additional attrs for svg
    ):
        "A `use` element in an `svg` element refering to `nm`"
        store_attr()
        self.attrs = sz_attrs(self.sz, self.vbox) | attrs
        style = _style_str(stroke, fill, stroke_width)
        self.uses = [Use(href=f"#{pre}{nm}", style=style, transform=trans)]
    
    def __ft__(self): return Svg(*self.uses, cls=self.cls, **self.attrs)

    def _repr_markdown_(self): return self.__ft__()._repr_markdown_()
    def __repr__(self): return self.__ft__().__repr__()
    def __str__(self): return self.__ft__().__str__()
    def __call__(self, *args, **kwargs): return self.__ft__().__call__(*args, **kwargs)

In [ ]:
accs = Icon('accessibility', 'l-', sz=16)
accs

```html
<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" height="16px" width="16px" class="lucide-icon"><use href="#l-accessibility"></use></svg>
```

In [ ]:
print(accs(foo='bar'))

<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" height="16px" width="16px" class="lucide-icon" foo="bar"><use href="#l-accessibility"></use></svg>


In [ ]:
show(accs)

In [ ]:
show(Icon('accessibility', 'l-', sz=16, stroke_width=3))

In [ ]:
#| export
@patch
def __iadd__(self:Icon, b):
    self.uses += b.uses
    return self

@patch
def __add__(self:Icon, b):
    res = deepcopy(self)
    res += b
    return res

In [ ]:
ad = Icon('a-arrow-down', 'l-')
show(accs+ad)

In [ ]:
sps = sprites(icons, [nm, 'accessibility', 'check', 'user', 'plus', 'circle', 'file', 'arrow-up'], 'l-')
show(sps)

In [ ]:
show(Icon('circle', 'l-', sz=24) + Icon('check', 'l-', stroke='red'))

In [ ]:
show(Icon('circle', 'l-', sz=24) + Icon('check', 'l-', stroke='red', trans='translate(2,3) scale(0.8)'))

In [ ]:
show(Icon('user', 'l-', sz=24, trans='translate(-4, 0)', stroke_width=1) +
     Icon('plus', 'l-', trans='translate(9, 3) scale(0.6)', stroke_width=3))

In [ ]:
show(Icon('file', 'l-', sz=24, fill='yellow') + 
     Icon('arrow-up', 'l-', trans='translate(2, 3) scale(0.7)'))

In [ ]:
#| export
def SvgStyle(cls="lucide-icon"):
    "Styles required for lucide icons to display correctly"
    return Style(f'.{cls} {{ stroke: currentColor; fill: none; stroke-width: 2; stroke-linecap: round; stroke-linejoin: round; }}')

In [ ]:
#| export
class SvgSprites:
    "Create an track used icons"
    def __init__(self, pre='', vbox=24, sz=24, cls="lucide-icon", nms=(), **attrs):
        nms = set(nms)
        self.attrs = attrs
        self.icons = read_icons()
        store_attr()

    def __call__(self, nm, sz=None, cls="", **kw):
        self.nms.add(nm)
        if not sz: sz=self.sz
        attrs = self.attrs | kw
        return Icon(nm, self.pre, vbox=self.vbox, sz=sz, cls = f"{self.cls} {cls}", **attrs)

    def __ft__(self):
        return SvgStyle(cls=self.cls),sprites(self.icons, self.nms, self.pre)

In [ ]:
ss = SvgSprites('lc-', sz=18)
ss(nm)

```html
<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" height="18px" width="18px" class="lucide-icon "><use href="#lc-a-arrow-down"></use></svg>
```

In [ ]:
show(ss('a-arrow-down'))
show(ss('accessibility', stroke_width=3))
show(ss)

In [ ]:
show(Style('.hover-bg:hover { background-color: aliceblue; }'))
show(ss('accessibility', sz=30, cls='hover-bg'))

In [ ]:
show(ss('accessibility', stroke='blue'))

In [ ]:
show(Div(ss('accessibility', stroke='blue'), style='background-color: aliceblue'))

In [ ]:
ss = SvgSprites('lc-', sz=32)
show(ss('circle', stroke='steelblue') + ss('x', stroke='crimson'))

In [ ]:
show(ss('arrow-up', trans='scale(0.7) translate(8,3)') +
     ss('zap', stroke='red', trans='scale(0.5) translate(2, 19)'))
show(ss)

In [ ]:
show(ss('arrow-down', trans='scale(0.7) translate(8,0)') +
     ss('zap', stroke='red', trans='scale(0.5) translate(2, 19)'))
show(ss)

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()